In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

import pickle

In [60]:
df = pd.read_csv("fullDataset.csv")


tconsts = df['tconst']
df.set_index('tconst', inplace=True)

#df = df.drop('primaryTitle')

numeric_cols = ["startYear", "runtimeMinutes", "rating"]
binary_cols = list(df.columns)
binary_cols.remove("startYear")
binary_cols.remove("runtimeMinutes")
binary_cols.remove("rating")
binary_cols.remove("primaryTitle")





In [61]:
print(binary_cols)

['primaryGenre_Adult', 'primaryGenre_Adventure', 'primaryGenre_Animation', 'primaryGenre_Biography', 'primaryGenre_Comedy', 'primaryGenre_Crime', 'primaryGenre_Documentary', 'primaryGenre_Drama', 'primaryGenre_Family', 'primaryGenre_Fantasy', 'primaryGenre_Film-Noir', 'primaryGenre_History', 'primaryGenre_Horror', 'primaryGenre_Music', 'primaryGenre_Musical', 'primaryGenre_Mystery', 'primaryGenre_News', 'primaryGenre_Reality-TV', 'primaryGenre_Romance', 'primaryGenre_Sci-Fi', 'primaryGenre_Science Fiction', 'primaryGenre_Short', 'primaryGenre_Sport', 'primaryGenre_TV Movie', 'primaryGenre_Thriller', 'primaryGenre_War', 'primaryGenre_Western', 'primaryGenre_\\N', 'actor1_', 'actor1_nm0000115', 'actor1_nm0000158', 'actor1_nm0000134', 'actor1_nm0001191', 'actor1_nm0000329', 'actor1_nm0000136', 'actor1_nm0000246', 'actor1_nm0000142', 'actor1_nm0000230', 'actor1_nm0000154', 'actor1_nm0000243', 'actor1_nm0000206', 'actor1_nm0000129', 'actor1_nm0000242', 'actor1_nm0000553', 'actor1_nm0000552'

In [62]:
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(df[numeric_cols])
scaled_numeric_df = pd.DataFrame(scaled_numeric, columns=numeric_cols, index=df.index)

In [63]:
df_features = pd.concat([scaled_numeric_df, df[binary_cols]], axis=1)


In [71]:
df_features.to_csv("scaled_dataset.csv", index=False)

In [68]:
def get_similar_movies_cosine(input_ids, per_movie_recommendations=2):
    recommendations = []

    for movie_id in input_ids:
        if movie_id not in df_features.index:
            continue

        vector = df_features.loc[movie_id].values.reshape(1, -1)

        # Compute cosine similarity between this movie and all others
        similarities = cosine_similarity(vector, df_features.values)[0]

        sim_series = pd.Series(similarities, index=df_features.index)
        sim_series = sim_series.drop(labels=[movie_id])  # remove self

        top_ids = sim_series.sort_values(ascending=False).head(per_movie_recommendations).index.tolist()
        recommendations.extend(top_ids)

    if not recommendations:
        raise ValueError("No valid recommendations could be made from the input IDs.")

    return recommendations

In [70]:
test_ids = ['tt0111161', 'tt0068646']  # The Shawshank Redemption & The Godfather
try:
    results = get_similar_movies_cosine(test_ids, per_movie_recommendations=2)

    print("Top similar movies:", results)
except ValueError as e:
    print("Error:", e)

Top similar movies: ['tt0137523', 'tt0073486', 'tt0110912', 'tt0086250']
